In [1]:
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
import numpy as np
import scipy.linalg as la
from qiskit_aer import AerSimulator
from qiskit.circuit import Gate
from matplotlib import pyplot as plt
import time
import os

In [2]:
def create_a(N_max: int) -> np.ndarray:
    a = np.zeros((N_max,) * 2)
    for i in range(N_max - 1):
        a[i, i + 1] = np.sqrt(i + 1)
    a_dag = a.T
    return a, a_dag

def create_H(N_max: int, lam: float) -> np.ndarray:
    a, a_dag = create_a(N_max)
    x = (a + a_dag) / np.sqrt(2)
    x4 = x @ x @ x @ x
    return a_dag @ a + 0.5 * np.eye(N_max) + lam * x4

def create_x(N_max: int) -> np.ndarray:
    a, a_dag = create_a(N_max)
    return (a + a_dag) / np.sqrt(2)

In [3]:
N_max = 4; lam = 0.1

In [4]:
H = create_H(N_max, lam)

In [5]:
H_op = SparsePauliOp.from_operator(H)

In [7]:
cII = 2.375
cIZ = -0.5
cXI = 0.28977775
cXZ = -0.07764571
cZI =  -1.15
cZZ = -0.15

In [8]:
def strang_step(dt: float) -> QuantumCircuit:
    qc = QuantumCircuit(2)
    # A half (Z-sector)
    qc.rz(2 * cZI * (dt/2), 1)
    qc.rz(2 * cIZ * (dt/2), 0)
    qc.rzz(2 * cZZ * (dt/2), 1, 0)
    # B full (X-sector)
    qc.h(1)
    qc.rz(2 * cXI * dt, 1)
    qc.rzz(2 * cXZ * dt, 1, 0)
    qc.h(1)
    # A half again
    qc.rz(2 * cZI * (dt/2), 1)
    qc.rz(2 * cIZ * (dt/2), 0)
    qc.rzz(2 * cZZ * (dt/2), 1, 0)
    qc.global_phase -= cII * dt
    return qc

def strang_step_k(dt: float, k: int, backend=None) -> Gate:
    U_pow = QuantumCircuit(2)
    U_dt  = strang_step(dt)
    for _ in range(int(k)):
        U_pow.compose(U_dt, inplace=True)
    # let merges happen before control
    # U_pow = transpile(U_pow, backend=backend, optimization_level=1)
    return U_pow.to_gate(label=f"U_strang({k}dt)")

def hadamard_test_re(dt: float, k: int, backend=None) -> QuantumCircuit:
    anc = QuantumRegister(1, 'a')
    sys = QuantumRegister(2, 'q')
    cre = ClassicalRegister(1, 'c')
    qc  = QuantumCircuit(anc, sys, cre)

    # prepare the initial state
    qc.h(sys[0])

    # Hadamard test (Re)
    qc.h(anc[0])
    U_pow_gate = strang_step_k(dt, k, backend=backend)
    CU_pow = U_pow_gate.control(1)
    qc.append(CU_pow, [anc[0], sys[0], sys[1]])

    qc.h(anc[0])
    qc.measure(anc[0], cre[0])
    return qc

def hadamard_test_im(dt: float, k: int, backend=None) -> QuantumCircuit:
    areg = QuantumRegister(1, 'a')
    qreg = QuantumRegister(2, 'q')
    creg = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(areg, qreg, creg)

    qc.h(qreg[0])

    qc.h(areg[0])
    U_pow_gate = strang_step_k(dt, k, backend=backend)
    CU_pow = U_pow_gate.control(1)
    qc.append(CU_pow, areg[:] + qreg[:])
    qc.sdg(areg[0]); qc.h(areg[0]); qc.measure(areg, creg)
    return qc

In [9]:
def sanity_check_step(dt: float) -> None:
    U_step_gate  = Operator(strang_step(dt)).data
    U_step_exact = la.expm(-1j * H * dt)
    step_err = np.linalg.norm(U_step_gate - U_step_exact, 2)
    w, _ = la.eig(U_step_gate)
    Ek_gate = np.mod(-np.angle(w)/dt, 2*np.pi/dt)
    print(f"||U_S(dt) - U_exact(dt)||_2 = {step_err:.3e}")
    print("gate-phase energies (sorted, first 4):", np.sort(Ek_gate)[:4])
    print("exact H energies (first 4):", np.sort(np.linalg.eigvalsh(H)).real[:4])

def exp_from_counts(counts, shots):
    p0 = counts.get('0', 0) / shots
    p1 = counts.get('1', 0) / shots
    return p0 - p1

In [10]:
def hann_window_index(k, N):
    return 0.5 * (1.0 - np.cos(2 * np.pi * k / (N - 1)))

In [11]:
LAM = 0.1
NT = 256
SHOTS = 5_000

WMIN, WMAX, WPTS = -2.5, 0.0, 2001

In [12]:
dt = 0.1

In [13]:
DT = 0.1

In [14]:
# to enable latex in plot
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

In [ ]:
# def run_noiseless_and_plot():
# build circuits
ks = np.arange(NT)
re_circs = [hadamard_test_re(dt, k) for k in ks]
im_circs = [hadamard_test_im(dt, k) for k in ks]

# Aer Backend
sim = AerSimulator()
tre = transpile(re_circs, sim, optimization_level=3, layout_method="trivial", seed_transpiler=123)
tim = transpile(im_circs, sim, optimization_level=3, layout_method="trivial", seed_transpiler=123)

res_re = sim.run(tre, shots=SHOTS, seed_simulator=777).result()
res_im = sim.run(tim, shots=SHOTS, seed_simulator=778).result()

# Reconstruct Ck
ReC = np.array([exp_from_counts(res_re.get_counts(i), SHOTS) for i in range(NT)])
ImC = np.array([exp_from_counts(res_im.get_counts(i), SHOTS) for i in range(NT)])
Ck  = ReC + 1j*ImC
meanCC = np.mean(Ck)

# Hann
w  = np.array([hann_window_index(k, NT) for k in range(NT)])
ks = np.arange(NT)
def FHann(wm):
    return np.sum((Ck - meanCC) * w * np.exp(-1j * wm * ks * DT))

wm_grid = np.linspace(WMIN, WMAX, WPTS)
absFH   = np.abs(np.array([FHann(wm) for wm in wm_grid]))

# exact energies for vertical guides
E_exact = np.sort(np.linalg.eigvalsh(H)).real

# Plot Hann-only with dashed −E_k guides
plt.figure(figsize=(9,5))


In [ ]:
plt.plot(wm_grid, absFH, linewidth=1.6, label=r'$|F_{\text{Hann}}(\omega)|$ (noiseless Strang)')
for Ek in E_exact[:4]:
    x = -Ek
    if WMIN <= x <= WMAX:
        plt.axvline(x, linestyle='--', linewidth=1.0, alpha=0.6)
plt.xlabel(r"$\omega$", size=15)
plt.ylabel(r"$\lvert F_{\text{Hann}}(\omega)\rvert$", size=15)
plt.savefig(r"./spectrum_extraction_lam_0p1_aho.png", dpi=600)

In [48]:
from dataclasses import dataclass
from qiskit_aer.noise import NoiseModel, ReadoutError, depolarizing_error, thermal_relaxation_error

@dataclass
class NoiseParams:
    # Readout confusion (meas)
    p01: float = 0.02    # P(1|0)
    p10: float = 0.03    # P(0|1)
    # Gate depolarizing (gate)
    p1q: float = 0.0010  # 1q depol
    p2q: float = 0.0060  # 2q depol (largest impact)
    # Decoherence (T1/T2) in ns + nominal gate durations
    T1:  float = 120e3
    T2:  float = 110e3
    t_sx: float = 35
    t_cx: float = 300
    p_th: float = 0.0    # excited-state population for thermal channel

def noise_noiseless():
    return None

def noise_meas(params: NoiseParams):
    nm = NoiseModel()
    rd = ReadoutError([[1-params.p01, params.p01],
                       [params.p10,   1-params.p10]])
    # Only ancilla is measured, but attaching to [0,1,2] is safe
    for q in [0,1,2]:
        nm.add_readout_error(rd, [q])
    return nm

def noise_meas_gate(params: NoiseParams):
    nm = noise_meas(params)
    dep1 = depolarizing_error(params.p1q, 1)
    dep2 = depolarizing_error(params.p2q, 2)
    for q in [0,1,2]:
        nm.add_quantum_error(dep1, ['sx','x','rz'], [q])
    for pair in ([0,1],[0,2],[1,2]):
        nm.add_quantum_error(dep2, ['cx'], pair)
    return nm

def noise_meas_gate_decoh(params: NoiseParams):
    nm = noise_meas(params)
    th1 = thermal_relaxation_error(params.T1, params.T2, params.t_sx, params.p_th)
    th2 = thermal_relaxation_error(params.T1, params.T2, params.t_cx, params.p_th).tensor(
          thermal_relaxation_error(params.T1, params.T2, params.t_cx, params.p_th))
    dep1 = depolarizing_error(params.p1q, 1)
    dep2 = depolarizing_error(params.p2q, 2)
    comb1 = th1.compose(dep1)   # model depol during gate + relaxation
    comb2 = th2.compose(dep2)
    for q in [0,1,2]:
        nm.add_quantum_error(comb1, ['sx','x','rz'], [q])
    for pair in ([0,1],[0,2],[1,2]):
        nm.add_quantum_error(comb2, ['cx'], pair)
    return nm

def run_mode_get_FHann(label, noise_model, NT, SHOTS, WMIN, WMAX, WPTS):
    print(f"\n=== Running: {label} ===")
    ks = np.arange(NT)
    re_circs = [hadamard_test_re(dt, k) for k in ks]
    im_circs = [hadamard_test_im(dt, k) for k in ks]
    basis = ['sx', 'x', 'rz', 'cx']

    sim = AerSimulator() if noise_model is None else AerSimulator(noise_model=noise_model)

    # fast transpile; keep seeds for reproducibility
    tre = transpile(re_circs, sim, optimization_level=1, layout_method="trivial", seed_transpiler=123)
    tim = transpile(im_circs, sim, optimization_level=1, layout_method="trivial", seed_transpiler=123)

    res_re = sim.run(tre, shots=SHOTS, seed_simulator=777).result()
    res_im = sim.run(tim, shots=SHOTS, seed_simulator=778).result()

    ReC = np.array([exp_from_counts(res_re.get_counts(i), SHOTS) for i in range(NT)])
    ImC = np.array([exp_from_counts(res_im.get_counts(i), SHOTS) for i in range(NT)])
    Ck  = ReC + 1j*ImC
    meanCC = np.mean(Ck)

    # Hann-window spectrum
    w  = 0.5*(1.0 - np.cos(2*np.pi * np.arange(NT)/(NT-1)))
    def FHann(wm):
        return np.sum((Ck - meanCC) * w * np.exp(-1j * wm * np.arange(NT) * DT))

    wm_grid = np.linspace(WMIN, WMAX, WPTS)
    absFH   = np.abs(np.array([FHann(wm) for wm in wm_grid]))
    return wm_grid, absFH

In [73]:
# --- overlay driver for four modes ---
def run_all_modes_and_overlay():
    params = NoiseParams()  # tweak here if you want gentler/harsher noise
    modes = [
        ("noiseless",           noise_noiseless()),
        ("meas",                noise_meas(params)),
        ("meas+gate",           noise_meas_gate(params)),
        ("meas+gate+decoh",     noise_meas_gate_decoh(params)),
    ]

    curves = []
    for label, nm in modes:
        wgrid, absFH = run_mode_get_FHann(label, nm, NT, SHOTS, WMIN, WMAX, WPTS)
        curves.append((label, wgrid, absFH))

    # exact energies for guides
    E_exact = np.sort(np.linalg.eigvalsh(H)).real

    # overlay plot
    plt.figure(figsize=(10,6))
    for label, wgrid, absFH in curves:
        plt.plot(wgrid, absFH, label=label, linewidth=1.5)
    for Ek in E_exact[:4]:
        x = -Ek
        if WMIN <= x <= WMAX:
            plt.axvline(x, linestyle='--', linewidth=1.0, alpha=0.55)
    plt.xlabel(r"$\omega$", size = 15); plt.ylabel(r"$|F_{\text{Hann}}(\omega)|$", size = 15)
    plt.title(r"Hann spectra overlay: $d=4, \lambda=0.1$", size = 18)
    plt.legend(loc='upper left'); plt.tight_layout()
    # plt.savefig('./plots/allmodes.png', dpi=300) # uncomment to save plot
    plt.show()

# =============================
# Run overlay (comment out if you only want noiseless)
# =============================
if __name__ == "__main__":
    start_time = time.time()
    run_all_modes_and_overlay()
    tot_time = time.time() - start_time
    print(f'Took {end_time} seconds in total')


=== Running: noiseless ===


KeyboardInterrupt: 